# Decision Trees
Decision trees are classifiers that parition space by forming a series of conditions. e.g. value is greater than X, class A otherwise Class B etc. etc.

To test this process we create data that is an overlapping spiral using a paramatic equation. The formation isn't important but we define parameters for number of arcs, noise and gap between spiral arms, which we use to see how the classifier behaves.

You can change these values and view the spiral output.

In [ ]:
import numpy as np
from numpy import pi
from numpy.random import randint
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, RocCurveDisplay
from sklearn.model_selection import cross_val_score, train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

### Data generator
Experiment with changing noise, test/train split and number of samples

In [ ]:
# generate spiral data (see https://gist.github.com/45deg/e731d9e7f478de134def5668324c44c5)
noise = 1.0  #  Size of the noise added.
N = 200      #  number of samples
split = 0.25 #  test train split i.e. 0.25 = 1/4 of the data used for testing.
gap = 2      #  gap between spiral arms
arcs = 1.0   #  number of rotations for sprial
theta = np.sqrt(np.random.rand(N))*arcs*2*pi # np.linspace(0,2*pi,100)

r_a = gap*theta + pi
data_a = np.array([np.cos(theta)*r_a, np.sin(theta)*r_a]).T
x_a = data_a + noise*np.random.randn(N,2)

r_b = -gap*theta - pi
data_b = np.array([np.cos(theta)*r_b, np.sin(theta)*r_b]).T
x_b = data_b + noise*np.random.randn(N,2)

res_a = np.append(x_a, np.zeros((N,1)), axis=1)
res_b = np.append(x_b, np.ones((N,1)), axis=1)

res = np.append(res_a, res_b, axis=0)
np.random.shuffle(res)

plt.scatter(x_a[:,0],x_a[:,1])
plt.scatter(x_b[:,0],x_b[:,1])
X=np.r_[x_a,x_b]
y=np.r_[np.zeros(N),np.ones(N)]
print("Input data")
plt.show()

X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=split, random_state=None)


## Basic Decision Tree
This creates a Decision Tree Classifier, which will be visualised as a set of conditional branches.
We also see a ROC curve (see lectures) which allows us to see how well it is doing.

The ROC curve is summarised with the Area Under the Curve (AUC), the closer this is to 1.0 the better the classfication.

We additionally perform a 10 fold Cross Validation to assess how well the classifier will work on new data.

Experiment with changing the maximum depth of the tree, how does this effect the AUC.
Does the average AUC predicted by cross validation agree with the value from the test data?

In [ ]:
max_depth = None # how deep the tree generated can go, None is uncapped, experiment with lower values

clf = DecisionTreeClassifier(random_state=0,max_depth=max_depth)

print("10 fold cross validation")
print(cross_val_score(clf, X_train,y_train, scoring='roc_auc', cv=10))

print("Final model")
clf = clf.fit(X_train,y_train)
print("AUC",roc_auc_score(y_test, clf.predict(X_test)))

clf_disp = RocCurveDisplay.from_estimator(clf, X_test, y_test)
plt.show()

print("Decision Tree")
plt.figure(figsize=(10,10))
plot_tree(clf)
plt.show()

### Decision boundary plot
This is a helper that shows the decision boundaries for the classifier, which we apply to the above tree.

How many parts does the tree have, do they all join up?

In [ ]:
def plot_decision_boundaries(_X, _y, tree):
    feature_1, feature_2 = np.meshgrid(
        np.linspace(_X[:, 0].min(), _X[:, 0].max()),
        np.linspace(_X[:, 1].min(), _X[:, 1].max())
    )
    grid = np.vstack([feature_1.ravel(), feature_2.ravel()]).T
    y_pred = np.reshape(tree.predict(grid), feature_1.shape)
    display = DecisionBoundaryDisplay(
        xx0=feature_1, xx1=feature_2, response=y_pred
    )
    display.plot(alpha=0.5, cmap="plasma")
    display.ax_.scatter(
        _X[:, 0], _X[:, 1], c=_y, edgecolor="black"
    )
    plt.show()
    
print("Decision Boundary Plot")
print("Training data")
plot_decision_boundaries(X_train, y_train, clf)
print("Test data")
plot_decision_boundaries(X_test, y_test, clf)

## Bagging - Random forest

Decision trees are prone to overfitting, as they keep separating data over and over again.
To Avoid this we can create lots of decision trees and then get a consensus result from them all.
This is known as a Random Forest.

This example creates a random forest from the data, we have some hyperparameters 9such as number of estimators and maximum depth, we perform a small randomised search to pick these values using the data and a 5-fold Cross Validation.


In [ ]:
# Hyperparameter dictionary
param_dist={
    'n_estimators':range(50,100), # size of forest
    'max_depth':range(1,20) # depth of forest - "pruning"
}

# perform a "randomised" grid search of hyperparameters
print("Picking hyperparameters")
rf = RandomForestClassifier()
rand_search=RandomizedSearchCV(rf,
                               param_distributions=param_dist,
                               n_iter=5, 
                               cv=5)
rand_search.fit(X_train, y_train)

# get best model and parameters
best_rf = rand_search.best_estimator_
print(rand_search.best_params_)

# evaluate best model
print("AUC", roc_auc_score(y_test, best_rf.predict(X_test)))

# print decision boundary plot.
print("Decision Boundary Plot")
print("Training data")
plot_decision_boundaries(X_train, y_train, best_rf)
print("Test data")
plot_decision_boundaries(X_test, y_test, best_rf)

## Boosting

Boosting creates a sequence of classifiers with a shallow decision tree (we set max_depth 4 to start with).
The points that aren't classified by this tree are then weighted and another tree is created to better classify these.
Again the series of trees are used to find the final result.

_Note_: The spiral dataset does not benefit as much from this approach.


In [ ]:
max_depth = 4

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(random_state=0,max_depth=max_depth), n_estimators=300, random_state=0
)

ada_clf.fit(X_train, y_train)

print("AUC", roc_auc_score(y_test, ada_clf.predict(X_test)))

# print decision boundary plot.
print("Decision Boundary Plot")
print("Training data")
plot_decision_boundaries(X_train, y_train, ada_clf)
print("Test data")
plot_decision_boundaries(X_test, y_test, ada_clf)